# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,10.97,89,44,1.97,NZ,1705485403
1,1,puerto baquerizo moreno,-0.9000,-89.6000,24.66,84,75,2.59,EC,1705485404
2,2,inirida,3.8653,-67.9239,25.02,80,83,0.39,CO,1705485405
3,3,avarua,-21.2078,-159.7750,26.03,83,40,5.66,CK,1705485405
4,4,badger,64.8000,-147.5333,-5.86,80,40,5.14,US,1705485406


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng",
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width=650, 
    frame_height=500, 
    size="Humidity", 
    color="City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_cond_df = city_data_df.loc[((city_data_df["Max Temp"] <= 30) & (city_data_df["Max Temp"]>=20)) & (city_data_df["Cloudiness"]==0),:]

# Drop any rows with null values
narrowed_cond_df=narrowed_cond_df.dropna(how="any")

# Display sample data
narrowed_cond_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
127,127,mawlaik,23.6333,94.4167,26.37,37,0,0.22,MM,1705485488
143,143,faya-largeau,17.9257,19.1043,21.94,13,0,6.17,TD,1705485498
171,171,bardai,11.6500,7.2333,23.31,13,0,7.69,NG,1705485516
173,173,hawaiian paradise park,19.5933,-154.9731,23.35,86,0,3.09,US,1705485517
207,207,myitkyina,25.3833,97.4000,24.59,47,0,0.72,MM,1705485540
209,209,katsuren-haebaru,26.3369,127.8719,22.43,52,0,7.72,JP,1705485541
245,245,vila velha,-20.3297,-40.2925,26.64,80,0,3.13,BR,1705485566
266,266,itacarambi,-15.1022,-44.0919,22.63,71,0,1.23,BR,1705485579
271,271,sao miguel do araguaia,-13.2750,-50.1628,23.72,85,0,2.16,BR,1705485583
276,276,rabigh,22.7986,39.0349,24.86,47,0,6.81,SA,1705485586


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:

# To remove warning 
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_cond_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

/var/folders/t5/g_jkh7097w5gq_dwkt0lw0qr0000gn/T/ipykernel_84741/815158723.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
127,mawlaik,MM,23.6333,94.4167,37,
143,faya-largeau,TD,17.9257,19.1043,13,
171,bardai,NG,11.6500,7.2333,13,
173,hawaiian paradise park,US,19.5933,-154.9731,86,
207,myitkyina,MM,25.3833,97.4000,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Print the API response
    print(name_address.text)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{"type":"FeatureCollection","features":[]}
mawlaik - nearest hotel: No hotel found
{"type":"FeatureCollection","features":[]}
faya-largeau - nearest hotel: No hotel found
{"type":"FeatureCollection","features":[]}
bardai - nearest hotel: No hotel found
{"type":"FeatureCollection","features":[]}
hawaiian paradise park - nearest hotel: No hotel found
{"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"New Light Hotel","country":"Myanmar","country_code":"mm","state":"Kachin State","city":"Myitkyina Township","municipality":"Myitkyina District","postcode":"10101","suburb":"မင်းရပ် ရပ်ကွက် Min Yat Ward","street":"သခင်နက်ဖေလမ်း Tha Khin Net Hpay Street","lon":97.40034,"lat":25.3841385,"formatted":"New Light Hotel, သခင်နက်ဖေလမ်း Tha Khin Net Hpay Street, Myitkyina Township, 10101, Myanmar","address_line1":"New Light Hotel","address_line2":"သခင်နက်ဖေလမ်း Tha Khin Net Hpay Street, Myitkyina Township, 10101, Myanmar","categories":["accommodation"

,City,Country,Lat,Lng,Humidity,Hotel Name
127,mawlaik,MM,23.6333,94.4167,37,No hotel found
143,faya-largeau,TD,17.9257,19.1043,13,No hotel found
171,bardai,NG,11.6500,7.2333,13,No hotel found
173,hawaiian paradise park,US,19.5933,-154.9731,86,No hotel found
207,myitkyina,MM,25.3833,97.4000,47,New Light Hotel
209,katsuren-haebaru,JP,26.3369,127.8719,52,No hotel found
245,vila velha,BR,-20.3297,-40.2925,80,Hotel Prainha
266,itacarambi,BR,-15.1022,-44.0919,71,No hotel found
271,sao miguel do araguaia,BR,-13.2750,-50.1628,85,Hotel São Miguel
276,rabigh,SA,22.7986,39.0349,47,S.A. TALKE Villa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map_markings = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map_markings

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)